In [2]:
import os
import sys
import numpy as np
import pandas as pd

from IPython.core.pylabtools import figsize
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline
figsize(12, 8)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('precision', 5)
pd.set_option('display.max_colwidth', 500) 

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn.preprocessing import scale

X, Y = make_moons(noise=0.2, random_state=0, n_samples=1000)
X = scale(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.5)

In [5]:
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.constraints import maxnorm 
from keras import backend as keras_backend 
from keras.utils import np_utils 
from keras.models import load_model 
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import cross_val_score 

random_seed = 42 
np.random.seed(random_seed)

/home/dan/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
X_train.shape

(500, 2)

2

In [40]:
keras_input_shape = X_train.shape[1]
number_of_classes = np.unique(y_train).shape[0]

In [41]:
# Based on Example From Deep Learning Vol 2
## Create a model_making function for use in Scikit learn
def make_model(number_of_layers=2, neurons_per_layer=32, dropout_ratio=0.2, optimizer='adam'):
    model = Sequential()
    # first layer is special, because it sets input_shape   
    model.add(Dense(neurons_per_layer, 
                    input_shape=[keras_input_shape], 
                    activation='relu', 
                    kernel_constraint=maxnorm(3)))    
    model.add(Dropout(dropout_ratio))    
    
    # now add in all the rest of the dense-dropout layers
    for i in range(number_of_layers-1):
        model.add(Dense(neurons_per_layer, activation='relu',
                        kernel_constraint=maxnorm(3)))
        model.add(Dropout(dropout_ratio))
    
    # finish up with a softmax layer with 10 outputs
    model.add(Dense(number_of_classes, activation='sigmoid'))
    
    # compile the model and return it    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [42]:
from keras.wrappers.scikit_learn import KerasClassifier  # can also use KerasRegressor

kc_model = KerasClassifier(build_fn=make_model,
                           # parameters for the model-making function
                           number_of_layers=3, neurons_per_layer=32,
                           optimizer = 'adam',
                           # parameters for scikit-learn
                           epochs=100, batch_size=256, verbose=2)

In [43]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=10, shuffle=True, 
                        random_state=random_seed)

from sklearn.model_selection import cross_val_score
results = cross_val_score(kc_model, X_train, y_train, cv=kfold, verbose=0)


Epoch 1/100
 - 1s - loss: 0.6891 - acc: 0.5590
Epoch 2/100
 - 0s - loss: 0.6768 - acc: 0.6281
Epoch 3/100
 - 0s - loss: 0.6693 - acc: 0.6347
Epoch 4/100
 - 0s - loss: 0.6614 - acc: 0.6748
Epoch 5/100
 - 0s - loss: 0.6470 - acc: 0.7105
Epoch 6/100
 - 0s - loss: 0.6360 - acc: 0.7416
Epoch 7/100
 - 0s - loss: 0.6240 - acc: 0.7617
Epoch 8/100
 - 0s - loss: 0.6200 - acc: 0.7595
Epoch 9/100
 - 0s - loss: 0.6005 - acc: 0.8062
Epoch 10/100
 - 0s - loss: 0.5950 - acc: 0.8018
Epoch 11/100
 - 0s - loss: 0.5789 - acc: 0.8241
Epoch 12/100
 - 0s - loss: 0.5639 - acc: 0.8330
Epoch 13/100
 - 0s - loss: 0.5416 - acc: 0.8441
Epoch 14/100
 - 0s - loss: 0.5368 - acc: 0.8619
Epoch 15/100
 - 0s - loss: 0.5104 - acc: 0.8552
Epoch 16/100
 - 0s - loss: 0.4872 - acc: 0.8664
Epoch 17/100
 - 0s - loss: 0.4700 - acc: 0.8731
Epoch 18/100
 - 0s - loss: 0.4606 - acc: 0.8597
Epoch 19/100
 - 0s - loss: 0.4360 - acc: 0.8664
Epoch 20/100
 - 0s - loss: 0.4204 - acc: 0.8619
Epoch 21/100
 - 0s - loss: 0.4036 - acc: 0.8775
E

 - 0s - loss: 0.2444 - acc: 0.8909
Epoch 73/100
 - 0s - loss: 0.2499 - acc: 0.8931
Epoch 74/100
 - 0s - loss: 0.2463 - acc: 0.9042
Epoch 75/100
 - 0s - loss: 0.2456 - acc: 0.8998
Epoch 76/100
 - 0s - loss: 0.2320 - acc: 0.9154
Epoch 77/100
 - 0s - loss: 0.2325 - acc: 0.8976
Epoch 78/100
 - 0s - loss: 0.2319 - acc: 0.8976
Epoch 79/100
 - 0s - loss: 0.2428 - acc: 0.9020
Epoch 80/100
 - 0s - loss: 0.2437 - acc: 0.9176
Epoch 81/100
 - 0s - loss: 0.2375 - acc: 0.8998
Epoch 82/100
 - 0s - loss: 0.2354 - acc: 0.9154
Epoch 83/100
 - 0s - loss: 0.2258 - acc: 0.9042
Epoch 84/100
 - 0s - loss: 0.2379 - acc: 0.8998
Epoch 85/100
 - 0s - loss: 0.2462 - acc: 0.9065
Epoch 86/100
 - 0s - loss: 0.2303 - acc: 0.9042
Epoch 87/100
 - 0s - loss: 0.2247 - acc: 0.9198
Epoch 88/100
 - 0s - loss: 0.2371 - acc: 0.9087
Epoch 89/100
 - 0s - loss: 0.2393 - acc: 0.9131
Epoch 90/100
 - 0s - loss: 0.2189 - acc: 0.9109
Epoch 91/100
 - 0s - loss: 0.2255 - acc: 0.9109
Epoch 92/100
 - 0s - loss: 0.2337 - acc: 0.9154
Epoch

Epoch 44/100
 - 0s - loss: 0.2564 - acc: 0.8931
Epoch 45/100
 - 0s - loss: 0.2573 - acc: 0.8953
Epoch 46/100
 - 0s - loss: 0.2580 - acc: 0.8909
Epoch 47/100
 - 0s - loss: 0.2598 - acc: 0.8909
Epoch 48/100
 - 0s - loss: 0.2656 - acc: 0.8931
Epoch 49/100
 - 0s - loss: 0.2640 - acc: 0.8886
Epoch 50/100
 - 0s - loss: 0.2750 - acc: 0.8976
Epoch 51/100
 - 0s - loss: 0.2576 - acc: 0.8998
Epoch 52/100
 - 0s - loss: 0.2597 - acc: 0.8998
Epoch 53/100
 - 0s - loss: 0.2608 - acc: 0.8909
Epoch 54/100
 - 0s - loss: 0.2478 - acc: 0.9020
Epoch 55/100
 - 0s - loss: 0.2438 - acc: 0.9042
Epoch 56/100
 - 0s - loss: 0.2468 - acc: 0.8864
Epoch 57/100
 - 0s - loss: 0.2397 - acc: 0.9020
Epoch 58/100
 - 0s - loss: 0.2529 - acc: 0.9042
Epoch 59/100
 - 0s - loss: 0.2376 - acc: 0.9020
Epoch 60/100
 - 0s - loss: 0.2507 - acc: 0.9020
Epoch 61/100
 - 0s - loss: 0.2452 - acc: 0.9020
Epoch 62/100
 - 0s - loss: 0.2398 - acc: 0.8998
Epoch 63/100
 - 0s - loss: 0.2441 - acc: 0.8953
Epoch 64/100
 - 0s - loss: 0.2348 - acc:

 - 0s - loss: 0.5784 - acc: 0.8537
Epoch 16/100
 - 0s - loss: 0.5670 - acc: 0.8758
Epoch 17/100
 - 0s - loss: 0.5527 - acc: 0.8780
Epoch 18/100
 - 0s - loss: 0.5389 - acc: 0.8692
Epoch 19/100
 - 0s - loss: 0.5256 - acc: 0.8714
Epoch 20/100
 - 0s - loss: 0.5130 - acc: 0.8714
Epoch 21/100
 - 0s - loss: 0.4918 - acc: 0.8780
Epoch 22/100
 - 0s - loss: 0.4853 - acc: 0.8803
Epoch 23/100
 - 0s - loss: 0.4593 - acc: 0.8780
Epoch 24/100
 - 0s - loss: 0.4330 - acc: 0.8914
Epoch 25/100
 - 0s - loss: 0.4273 - acc: 0.8803
Epoch 26/100
 - 0s - loss: 0.4028 - acc: 0.8803
Epoch 27/100
 - 0s - loss: 0.3925 - acc: 0.8825
Epoch 28/100
 - 0s - loss: 0.3800 - acc: 0.8891
Epoch 29/100
 - 0s - loss: 0.3511 - acc: 0.8914
Epoch 30/100
 - 0s - loss: 0.3418 - acc: 0.8825
Epoch 31/100
 - 0s - loss: 0.3313 - acc: 0.8891
Epoch 32/100
 - 0s - loss: 0.3203 - acc: 0.8891
Epoch 33/100
 - 0s - loss: 0.3195 - acc: 0.8980
Epoch 34/100
 - 0s - loss: 0.3012 - acc: 0.8958
Epoch 35/100
 - 0s - loss: 0.2964 - acc: 0.8958
Epoch

Epoch 87/100
 - 0s - loss: 0.2183 - acc: 0.9180
Epoch 88/100
 - 0s - loss: 0.2231 - acc: 0.9202
Epoch 89/100
 - 0s - loss: 0.2239 - acc: 0.9202
Epoch 90/100
 - 0s - loss: 0.2205 - acc: 0.9180
Epoch 91/100
 - 0s - loss: 0.2258 - acc: 0.9157
Epoch 92/100
 - 0s - loss: 0.2204 - acc: 0.9224
Epoch 93/100
 - 0s - loss: 0.2189 - acc: 0.9290
Epoch 94/100
 - 0s - loss: 0.2194 - acc: 0.9091
Epoch 95/100
 - 0s - loss: 0.2221 - acc: 0.9157
Epoch 96/100
 - 0s - loss: 0.2303 - acc: 0.9224
Epoch 97/100
 - 0s - loss: 0.2218 - acc: 0.9313
Epoch 98/100
 - 0s - loss: 0.2286 - acc: 0.9224
Epoch 99/100
 - 0s - loss: 0.2219 - acc: 0.9246
Epoch 100/100
 - 0s - loss: 0.2143 - acc: 0.9313
Epoch 1/100
 - 1s - loss: 0.7046 - acc: 0.4634
Epoch 2/100
 - 0s - loss: 0.6928 - acc: 0.4812
Epoch 3/100
 - 0s - loss: 0.6851 - acc: 0.4900
Epoch 4/100
 - 0s - loss: 0.6703 - acc: 0.4989
Epoch 5/100
 - 0s - loss: 0.6637 - acc: 0.5122
Epoch 6/100
 - 0s - loss: 0.6471 - acc: 0.5543
Epoch 7/100
 - 0s - loss: 0.6421 - acc: 0.583

 - 0s - loss: 0.2176 - acc: 0.9091
Epoch 59/100
 - 0s - loss: 0.2214 - acc: 0.9180
Epoch 60/100
 - 0s - loss: 0.2386 - acc: 0.9135
Epoch 61/100
 - 0s - loss: 0.2219 - acc: 0.9157
Epoch 62/100
 - 0s - loss: 0.2256 - acc: 0.9024
Epoch 63/100
 - 0s - loss: 0.2329 - acc: 0.9180
Epoch 64/100
 - 0s - loss: 0.2187 - acc: 0.9180
Epoch 65/100
 - 0s - loss: 0.2088 - acc: 0.9180
Epoch 66/100
 - 0s - loss: 0.2236 - acc: 0.9202
Epoch 67/100
 - 0s - loss: 0.2136 - acc: 0.9313
Epoch 68/100
 - 0s - loss: 0.2185 - acc: 0.9180
Epoch 69/100
 - 0s - loss: 0.2177 - acc: 0.9180
Epoch 70/100
 - 0s - loss: 0.2095 - acc: 0.9180
Epoch 71/100
 - 0s - loss: 0.2281 - acc: 0.9180
Epoch 72/100
 - 0s - loss: 0.2150 - acc: 0.9157
Epoch 73/100
 - 0s - loss: 0.2162 - acc: 0.9246
Epoch 74/100
 - 0s - loss: 0.2251 - acc: 0.9224
Epoch 75/100
 - 0s - loss: 0.2058 - acc: 0.9202
Epoch 76/100
 - 0s - loss: 0.2002 - acc: 0.9313
Epoch 77/100
 - 0s - loss: 0.2178 - acc: 0.9313
Epoch 78/100
 - 0s - loss: 0.2209 - acc: 0.9335
Epoch

In [44]:
print('results = {}\nresults.mean = {}'.format(results, results.mean()))

results = [1.         0.94117647 0.84313726 0.90196079 0.96078432 0.89795917
 0.89795917 0.93877554 0.87755102 0.93877554]
results.mean = 0.9198079288005829


In [46]:
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import MinMaxScaler 

estimators = [] 
estimators.append(('normalize_step', MinMaxScaler())) 
estimators.append(('model_step', kc_model)) 
pipeline = Pipeline(estimators)

In [50]:
from sklearn.model_selection import GridSearchCV

param_grid = dict(
    model_step__number_of_layers=[2, 3, 4], 
    model_step__neurons_per_layer=[ 20, 30, 40 ], 
    model_step__optimizer=[ 'adam', 'adadelta']
) 

grid_searcher1 = GridSearchCV(estimator=pipeline, param_grid=param_grid, verbose=2) 

search_results1 = grid_searcher1.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] model_step__neurons_per_layer=20, model_step__number_of_layers=2 
Epoch 1/100
 - 1s - loss: 0.6931 - acc: 0.5105
Epoch 2/100
 - 0s - loss: 0.6901 - acc: 0.5435
Epoch 3/100
 - 0s - loss: 0.6846 - acc: 0.5616
Epoch 4/100
 - 0s - loss: 0.6847 - acc: 0.5616
Epoch 5/100
 - 0s - loss: 0.6802 - acc: 0.5946
Epoch 6/100
 - 0s - loss: 0.6795 - acc: 0.5766
Epoch 7/100
 - 0s - loss: 0.6771 - acc: 0.5586
Epoch 8/100
 - 0s - loss: 0.6748 - acc: 0.5766
Epoch 9/100
 - 0s - loss: 0.6721 - acc: 0.5646
Epoch 10/100
 - 0s - loss: 0.6733 - acc: 0.5766
Epoch 11/100
 - 0s - loss: 0.6706 - acc: 0.5796
Epoch 12/100
 - 0s - loss: 0.6691 - acc: 0.5916
Epoch 13/100
 - 0s - loss: 0.6640 - acc: 0.5946
Epoch 14/100
 - 0s - loss: 0.6613 - acc: 0.6006
Epoch 15/100
 - 0s - loss: 0.6598 - acc: 0.6126
Epoch 16/100
 - 0s - loss: 0.6554 - acc: 0.6156
Epoch 17/100
 - 0s - loss: 0.6535 - acc: 0.5916
Epoch 18/100
 - 0s - loss: 0.6520 - acc: 0.6426
Epoch 19/100
 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.9s remaining:    0.0s


Epoch 1/100
 - 1s - loss: 0.7028 - acc: 0.4625
Epoch 2/100
 - 0s - loss: 0.6980 - acc: 0.4655
Epoch 3/100
 - 0s - loss: 0.6884 - acc: 0.5465
Epoch 4/100
 - 0s - loss: 0.6885 - acc: 0.5796
Epoch 5/100
 - 0s - loss: 0.6920 - acc: 0.5135
Epoch 6/100
 - 0s - loss: 0.6884 - acc: 0.5646
Epoch 7/100
 - 0s - loss: 0.6882 - acc: 0.5706
Epoch 8/100
 - 0s - loss: 0.6838 - acc: 0.5646
Epoch 9/100
 - 0s - loss: 0.6720 - acc: 0.6366
Epoch 10/100
 - 0s - loss: 0.6788 - acc: 0.5976
Epoch 11/100
 - 0s - loss: 0.6649 - acc: 0.6336
Epoch 12/100
 - 0s - loss: 0.6652 - acc: 0.6456
Epoch 13/100
 - 0s - loss: 0.6715 - acc: 0.6096
Epoch 14/100
 - 0s - loss: 0.6613 - acc: 0.6126
Epoch 15/100
 - 0s - loss: 0.6543 - acc: 0.6396
Epoch 16/100
 - 0s - loss: 0.6498 - acc: 0.6276
Epoch 17/100
 - 0s - loss: 0.6553 - acc: 0.6276
Epoch 18/100
 - 0s - loss: 0.6538 - acc: 0.6276
Epoch 19/100
 - 0s - loss: 0.6446 - acc: 0.6697
Epoch 20/100
 - 0s - loss: 0.6456 - acc: 0.6426
Epoch 21/100
 - 0s - loss: 0.6372 - acc: 0.6577
E

Epoch 69/100
 - 0s - loss: 0.4753 - acc: 0.8443
Epoch 70/100
 - 0s - loss: 0.4691 - acc: 0.8293
Epoch 71/100
 - 0s - loss: 0.4621 - acc: 0.8383
Epoch 72/100
 - 0s - loss: 0.4576 - acc: 0.8713
Epoch 73/100
 - 0s - loss: 0.4510 - acc: 0.8623
Epoch 74/100
 - 0s - loss: 0.4562 - acc: 0.8473
Epoch 75/100
 - 0s - loss: 0.4359 - acc: 0.8623
Epoch 76/100
 - 0s - loss: 0.4400 - acc: 0.8593
Epoch 77/100
 - 0s - loss: 0.4304 - acc: 0.8473
Epoch 78/100
 - 0s - loss: 0.4352 - acc: 0.8533
Epoch 79/100
 - 0s - loss: 0.4333 - acc: 0.8713
Epoch 80/100
 - 0s - loss: 0.4364 - acc: 0.8593
Epoch 81/100
 - 0s - loss: 0.4274 - acc: 0.8623
Epoch 82/100
 - 0s - loss: 0.4228 - acc: 0.8683
Epoch 83/100
 - 0s - loss: 0.4121 - acc: 0.8802
Epoch 84/100
 - 0s - loss: 0.4279 - acc: 0.8443
Epoch 85/100
 - 0s - loss: 0.4147 - acc: 0.8533
Epoch 86/100
 - 0s - loss: 0.4063 - acc: 0.8593
Epoch 87/100
 - 0s - loss: 0.4015 - acc: 0.8713
Epoch 88/100
 - 0s - loss: 0.4044 - acc: 0.8743
Epoch 89/100
 - 0s - loss: 0.3714 - acc:

Epoch 34/100
 - 0s - loss: 0.5414 - acc: 0.7748
Epoch 35/100
 - 0s - loss: 0.5448 - acc: 0.7718
Epoch 36/100
 - 0s - loss: 0.5301 - acc: 0.8048
Epoch 37/100
 - 0s - loss: 0.5112 - acc: 0.7988
Epoch 38/100
 - 0s - loss: 0.5065 - acc: 0.8138
Epoch 39/100
 - 0s - loss: 0.5045 - acc: 0.8138
Epoch 40/100
 - 0s - loss: 0.4904 - acc: 0.8138
Epoch 41/100
 - 0s - loss: 0.4946 - acc: 0.8198
Epoch 42/100
 - 0s - loss: 0.4645 - acc: 0.8318
Epoch 43/100
 - 0s - loss: 0.4719 - acc: 0.8138
Epoch 44/100
 - 0s - loss: 0.4937 - acc: 0.7898
Epoch 45/100
 - 0s - loss: 0.4647 - acc: 0.8048
Epoch 46/100
 - 0s - loss: 0.4399 - acc: 0.8138
Epoch 47/100
 - 0s - loss: 0.4507 - acc: 0.8258
Epoch 48/100
 - 0s - loss: 0.4115 - acc: 0.8529
Epoch 49/100
 - 0s - loss: 0.4163 - acc: 0.8529
Epoch 50/100
 - 0s - loss: 0.4065 - acc: 0.8529
Epoch 51/100
 - 0s - loss: 0.4103 - acc: 0.8108
Epoch 52/100
 - 0s - loss: 0.4233 - acc: 0.8378
Epoch 53/100
 - 0s - loss: 0.4198 - acc: 0.8348
Epoch 54/100
 - 0s - loss: 0.3885 - acc:

...

Epoch 69/100
 - 0s - loss: 0.3103 - acc: 0.8559
Epoch 70/100
 - 0s - loss: 0.2824 - acc: 0.8769
Epoch 71/100
 - 0s - loss: 0.2761 - acc: 0.8859
Epoch 72/100
 - 0s - loss: 0.2714 - acc: 0.8769
Epoch 73/100
 - 0s - loss: 0.3053 - acc: 0.8679
Epoch 74/100
 - 0s - loss: 0.2665 - acc: 0.8859
Epoch 75/100
 - 0s - loss: 0.2901 - acc: 0.8769
Epoch 76/100
 - 0s - loss: 0.2907 - acc: 0.8709
Epoch 77/100
 - 0s - loss: 0.2675 - acc: 0.8769
Epoch 78/100
 - 0s - loss: 0.2913 - acc: 0.8709
Epoch 79/100
 - 0s - loss: 0.2675 - acc: 0.8889
Epoch 80/100
 - 0s - loss: 0.2765 - acc: 0.8739
Epoch 81/100
 - 0s - loss: 0.2493 - acc: 0.9009
Epoch 82/100
 - 0s - loss: 0.2666 - acc: 0.8859
Epoch 83/100
 - 0s - loss: 0.2837 - acc: 0.8919
Epoch 84/100
 - 0s - loss: 0.2639 - acc: 0.8859
Epoch 85/100
 - 0s - loss: 0.2669 - acc: 0.8859
Epoch 86/100
 - 0s - loss: 0.2629 - acc: 0.9009
Epoch 87/100
 - 0s - loss: 0.2680 - acc: 0.8829
Epoch 88/100
 - 0s - loss: 0.2524 - acc: 0.8979
Epoch 89/100
 - 0s - loss: 0.2862 - acc:

Epoch 34/100
 - 0s - loss: 0.3620 - acc: 0.8529
Epoch 35/100
 - 0s - loss: 0.3421 - acc: 0.8619
Epoch 36/100
 - 0s - loss: 0.3528 - acc: 0.8679
Epoch 37/100
 - 0s - loss: 0.3395 - acc: 0.8589
Epoch 38/100
 - 0s - loss: 0.3691 - acc: 0.8529
Epoch 39/100
 - 0s - loss: 0.3438 - acc: 0.8739
Epoch 40/100
 - 0s - loss: 0.3261 - acc: 0.8709
Epoch 41/100
 - 0s - loss: 0.3058 - acc: 0.8799
Epoch 42/100
 - 0s - loss: 0.3346 - acc: 0.8709
Epoch 43/100
 - 0s - loss: 0.2984 - acc: 0.8739
Epoch 44/100
 - 0s - loss: 0.3535 - acc: 0.8559
Epoch 45/100
 - 0s - loss: 0.3289 - acc: 0.8649
Epoch 46/100
 - 0s - loss: 0.3231 - acc: 0.8709
Epoch 47/100
 - 0s - loss: 0.3355 - acc: 0.8709
Epoch 48/100
 - 0s - loss: 0.3218 - acc: 0.8739
Epoch 49/100
 - 0s - loss: 0.2923 - acc: 0.8739
Epoch 50/100
 - 0s - loss: 0.3558 - acc: 0.8769
Epoch 51/100
 - 0s - loss: 0.2950 - acc: 0.8799
Epoch 52/100
 - 0s - loss: 0.3224 - acc: 0.8739
Epoch 53/100
 - 0s - loss: 0.3023 - acc: 0.8769
Epoch 54/100
 - 0s - loss: 0.3567 - acc:

Epoch 1/100
 - 2s - loss: 0.6993 - acc: 0.4850
Epoch 2/100
 - 0s - loss: 0.6950 - acc: 0.4850
Epoch 3/100
 - 0s - loss: 0.6912 - acc: 0.4970
Epoch 4/100
 - 0s - loss: 0.6876 - acc: 0.5150
Epoch 5/100
 - 0s - loss: 0.6827 - acc: 0.5359
Epoch 6/100
 - 0s - loss: 0.6798 - acc: 0.5719
Epoch 7/100
 - 0s - loss: 0.6770 - acc: 0.5749
Epoch 8/100
 - 0s - loss: 0.6727 - acc: 0.6317
Epoch 9/100
 - 0s - loss: 0.6688 - acc: 0.6407
Epoch 10/100
 - 0s - loss: 0.6608 - acc: 0.6527
Epoch 11/100
 - 0s - loss: 0.6566 - acc: 0.7036
Epoch 12/100
 - 0s - loss: 0.6481 - acc: 0.7275
Epoch 13/100
 - 0s - loss: 0.6368 - acc: 0.7186
Epoch 14/100
 - 0s - loss: 0.6296 - acc: 0.7275
Epoch 15/100
 - 0s - loss: 0.6178 - acc: 0.7275
Epoch 16/100
 - 0s - loss: 0.6108 - acc: 0.7545
Epoch 17/100
 - 0s - loss: 0.6015 - acc: 0.7335
Epoch 18/100
 - 0s - loss: 0.5890 - acc: 0.7575
Epoch 19/100
 - 0s - loss: 0.5742 - acc: 0.7844
Epoch 20/100
 - 0s - loss: 0.5584 - acc: 0.7934
Epoch 21/100
 - 0s - loss: 0.5362 - acc: 0.8024
E

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  2.5min finished


Epoch 1/100
 - 2s - loss: 0.6889 - acc: 0.5200
Epoch 2/100
 - 0s - loss: 0.6866 - acc: 0.5480
Epoch 3/100
 - 0s - loss: 0.6800 - acc: 0.6480
Epoch 4/100
 - 0s - loss: 0.6766 - acc: 0.7320
Epoch 5/100
 - 0s - loss: 0.6720 - acc: 0.7680
Epoch 6/100
 - 0s - loss: 0.6705 - acc: 0.8040
Epoch 7/100
 - 0s - loss: 0.6645 - acc: 0.8140
Epoch 8/100
 - 0s - loss: 0.6620 - acc: 0.8320
Epoch 9/100
 - 0s - loss: 0.6571 - acc: 0.8360
Epoch 10/100
 - 0s - loss: 0.6484 - acc: 0.8400
Epoch 11/100
 - 0s - loss: 0.6454 - acc: 0.8220
Epoch 12/100
 - 0s - loss: 0.6374 - acc: 0.8500
Epoch 13/100
 - 0s - loss: 0.6306 - acc: 0.8480
Epoch 14/100
 - 0s - loss: 0.6229 - acc: 0.8540
Epoch 15/100
 - 0s - loss: 0.6111 - acc: 0.8520
Epoch 16/100
 - 0s - loss: 0.6086 - acc: 0.8520
Epoch 17/100
 - 0s - loss: 0.5951 - acc: 0.8620
Epoch 18/100
 - 0s - loss: 0.5868 - acc: 0.8600
Epoch 19/100
 - 0s - loss: 0.5780 - acc: 0.8660
Epoch 20/100
 - 0s - loss: 0.5638 - acc: 0.8580
Epoch 21/100
 - 0s - loss: 0.5563 - acc: 0.8400
E

In [52]:
best_model = grid_searcher1.best_estimator_

In [53]:
model_predictions = best_model.predict(X_test)

In [60]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score

cm = confusion_matrix(y_test, model_predictions)

cm_df = pd.DataFrame(data=cm, columns=['False (Predicted)', 'True (Predicted)'], index=['False', 'True'])
_numerator = (cm_df.loc['False', 'False (Predicted)'] + cm_df.loc['True', 'True (Predicted)'])
status = str(cm_df) + '\n\n' + 'Accuracy: ' + str(round((_numerator * 1.0 / cm_df.sum().sum()), 3))

scores = cross_val_score(best_model, X_train, y_train, cv=5, verbose=0)
status += '\nAverage crossval accuracy: {0} +/- {1}'.format(round(np.mean(scores), 3), round(np.std(scores), 3))
status += '\n All scores ' + str(scores) + '\n'

cf_report = classification_report(y_test, model_predictions)
status += '\n\n Classification Report:\n' + str(cf_report)
print(status)

Epoch 1/100
 - 2s - loss: 0.6762 - acc: 0.7300
Epoch 2/100
 - 0s - loss: 0.6739 - acc: 0.7400
Epoch 3/100
 - 0s - loss: 0.6686 - acc: 0.7600
Epoch 4/100
 - 0s - loss: 0.6650 - acc: 0.7575
Epoch 5/100
 - 0s - loss: 0.6600 - acc: 0.8100
Epoch 6/100
 - 0s - loss: 0.6591 - acc: 0.7875
Epoch 7/100
 - 0s - loss: 0.6509 - acc: 0.8175
Epoch 8/100
 - 0s - loss: 0.6450 - acc: 0.8475
Epoch 9/100
 - 0s - loss: 0.6387 - acc: 0.8300
Epoch 10/100
 - 0s - loss: 0.6356 - acc: 0.8450
Epoch 11/100
 - 0s - loss: 0.6243 - acc: 0.8550
Epoch 12/100
 - 0s - loss: 0.6228 - acc: 0.8500
Epoch 13/100
 - 0s - loss: 0.6132 - acc: 0.8600
Epoch 14/100
 - 0s - loss: 0.6095 - acc: 0.8600
Epoch 15/100
 - 0s - loss: 0.5952 - acc: 0.8450
Epoch 16/100
 - 0s - loss: 0.5885 - acc: 0.8675
Epoch 17/100
 - 0s - loss: 0.5814 - acc: 0.8700
Epoch 18/100
 - 0s - loss: 0.5738 - acc: 0.8650
Epoch 19/100
 - 0s - loss: 0.5608 - acc: 0.8725
Epoch 20/100
 - 0s - loss: 0.5527 - acc: 0.8675
Epoch 21/100
 - 0s - loss: 0.5483 - acc: 0.8675
E

 - 0s - loss: 0.3014 - acc: 0.8650
Epoch 73/100
 - 0s - loss: 0.2973 - acc: 0.8650
Epoch 74/100
 - 0s - loss: 0.3038 - acc: 0.8625
Epoch 75/100
 - 0s - loss: 0.2959 - acc: 0.8750
Epoch 76/100
 - 0s - loss: 0.3137 - acc: 0.8750
Epoch 77/100
 - 0s - loss: 0.3107 - acc: 0.8550
Epoch 78/100
 - 0s - loss: 0.3105 - acc: 0.8700
Epoch 79/100
 - 0s - loss: 0.3014 - acc: 0.8725
Epoch 80/100
 - 0s - loss: 0.3005 - acc: 0.8700
Epoch 81/100
 - 0s - loss: 0.3083 - acc: 0.8725
Epoch 82/100
 - 0s - loss: 0.2884 - acc: 0.8825
Epoch 83/100
 - 0s - loss: 0.3007 - acc: 0.8725
Epoch 84/100
 - 0s - loss: 0.2886 - acc: 0.8825
Epoch 85/100
 - 0s - loss: 0.3008 - acc: 0.8600
Epoch 86/100
 - 0s - loss: 0.2926 - acc: 0.8550
Epoch 87/100
 - 0s - loss: 0.2777 - acc: 0.8775
Epoch 88/100
 - 0s - loss: 0.3012 - acc: 0.8800
Epoch 89/100
 - 0s - loss: 0.2917 - acc: 0.8775
Epoch 90/100
 - 0s - loss: 0.2910 - acc: 0.8800
Epoch 91/100
 - 0s - loss: 0.2937 - acc: 0.8700
Epoch 92/100
 - 0s - loss: 0.3231 - acc: 0.8625
Epoch

Epoch 44/100
 - 0s - loss: 0.3098 - acc: 0.8800
Epoch 45/100
 - 0s - loss: 0.3075 - acc: 0.8925
Epoch 46/100
 - 0s - loss: 0.3136 - acc: 0.8525
Epoch 47/100
 - 0s - loss: 0.3037 - acc: 0.8750
Epoch 48/100
 - 0s - loss: 0.3047 - acc: 0.8775
Epoch 49/100
 - 0s - loss: 0.2833 - acc: 0.8825
Epoch 50/100
 - 0s - loss: 0.2843 - acc: 0.8900
Epoch 51/100
 - 0s - loss: 0.2861 - acc: 0.8875
Epoch 52/100
 - 0s - loss: 0.2781 - acc: 0.8775
Epoch 53/100
 - 0s - loss: 0.2855 - acc: 0.8725
Epoch 54/100
 - 0s - loss: 0.2790 - acc: 0.8825
Epoch 55/100
 - 0s - loss: 0.2760 - acc: 0.8725
Epoch 56/100
 - 0s - loss: 0.2746 - acc: 0.8825
Epoch 57/100
 - 0s - loss: 0.2914 - acc: 0.8700
Epoch 58/100
 - 0s - loss: 0.2911 - acc: 0.8600
Epoch 59/100
 - 0s - loss: 0.2794 - acc: 0.8750
Epoch 60/100
 - 0s - loss: 0.2777 - acc: 0.8825
Epoch 61/100
 - 0s - loss: 0.2868 - acc: 0.8925
Epoch 62/100
 - 0s - loss: 0.2693 - acc: 0.8875
Epoch 63/100
 - 0s - loss: 0.2744 - acc: 0.8775
Epoch 64/100
 - 0s - loss: 0.2700 - acc:

In [61]:
print(status)

       False (Predicted)  True (Predicted)
False                191                44
True                  38               227

Accuracy: 0.836
Average crossval accuracy: 0.89 +/- 0.028
 All scores [0.88999999 0.92000002 0.88999999 0.83999997 0.91000003]


 Classification Report:
             precision    recall  f1-score   support

          0       0.83      0.81      0.82       235
          1       0.84      0.86      0.85       265

avg / total       0.84      0.84      0.84       500

